# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Intellectual-Disability'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [25]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [27]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [28]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Intellectual-Disability"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE174555', 'GSE182101', 'GSE158385', 'GSE200864', 'GSE182081', 'GSE192767']

Repeat the below steps for all the accession numbers

In [41]:
cohort = accession_num = "GSE192767"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Intellectual-Disability/GSE192767/GSE192767_family.soft.gz',
 '/home/yxcheng/GEO/Intellectual-Disability/GSE192767/GSE192767_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [42]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Expression data from human lymphoblastoid cell lines (LCLs)"
!Series_summary	"The X-linked alpha thalassaemia intellectual disability syndrome (ATRX) protein is a member of the SWI/SNF family of chromatin remodelling factors which acts as an ATP dependent molecular motor. Germline mutations in ATRX give rise to a severe form of syndromal intellectual disability (ATR-X syndrome). To date, only a small number of genes have been identified that are affected by pathogenic ATRX mutations in human."
!Series_summary	"We performed microarray experiments on LCLs  from normal individuals and  patients with diverse pathogenic ATRX mutations, to identify more genes regulated by ATRX."
!Series_overall_design	"We used 20 LCLs  from unaffected individuals and  28 LCLs from patients for RNA extraction and hybridization on Affymetrix microarrays."


In [43]:
clinical_data.head()

,!Sample_geo_accession,GSM5765052,GSM5765053,GSM5765054,GSM5765055,GSM5765056,GSM5765057,GSM5765058,GSM5765059,GSM5765060,...,GSM5765090,GSM5765091,GSM5765092,GSM5765093,GSM5765094,GSM5765095,GSM5765096,GSM5765097,GSM5765098,GSM5765099
0,!Sample_characteristics_ch1,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,phenotype: ATR-X syndrome,...,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected,phenotype: unaffected
1,!Sample_characteristics_ch1,cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),...,cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL),cell type: human lymphoblastoid cell line (LCL)


In [40]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['age: 6.8 years',
  'age: 3 years',
  'age: 1.8 years',
  'age: 4 years',
  'age: 2 years',
  'age: 1 years',
  'age: 2.6 years',
  'age: 1.7 years',
  'age: 5 years',
  'age: 2.3 years',
  'age: 4.7 years',
  'age: 8 years',
  'age: 3.4 years',
  'age: 1 month',
  'age: 6 months',
  'age: 29 days',
  'age: 6 years',
  'age: 1.5 years',
  'age: 10 months',
  'age: 3.6 years',
  'age: 8 months',
  'age: 3 months',
  'age: 9.5 years',
  'age: 4.5 years',
  'age: 16 years',
  'age: 5.9 years',
  'age: 7 months',
  'age: 12 years',
  'age: 2.5 years',
  'age: 5.6 years'],
 1: ['gender: Male', 'gender: Female'],
 2: ['phenotype: Developmental delay and Intellectual Disability, Congenital Heart Disease.',
  'phenotype: Developmental delay and Central Hypotonia, Microcephaly',
  'phenotype: Developmental delay and Congenital heart disease, Dysmorphic Feature',
  'phenotype: Developmental delay and Delayed Speech, Dysmorphic Features',
  'phenotype: Developmental delay and Dysmorphic Feat

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [33]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Psoriasis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Psoriasis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an intege

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [10]:
is_gene_availabe = True
trait_row = 3
age_row = 1
gender_row = 0

trait_type = 'binary'

In [11]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [12]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'disease state: Down syndrome (Trisomy 21)':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('mos.'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, 1)

In [13]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:208: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM5319587,GSM5319588,GSM5319589,GSM5319590,GSM5319591,GSM5319592,GSM5319593,GSM5319594,GSM5319595,GSM5319596,...,GSM5319611,GSM5319612,GSM5319613,GSM5319614,GSM5319615,GSM5319616,GSM5319617,GSM5319618,GSM5319619,GSM5319620
Intellectual-Disability,0,1,0,0,1,0,0,1,0,1,...,0,0,1,1,0,0,0,1,1,0
Age,37,None,38,38,25,30,39,None,42,23,...,41,21,30,10,27,27,15,54,39,39
Gender,1,1,1,0,0,1,1,1,1,1,...,1,1,0,1,0,0,1,0,1,0


### Genetic data preprocessing and final filtering

In [14]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5319587,GSM5319588,GSM5319589,GSM5319590,GSM5319591,GSM5319592,GSM5319593,GSM5319594,GSM5319595,GSM5319596,...,GSM5319611,GSM5319612,GSM5319613,GSM5319614,GSM5319615,GSM5319616,GSM5319617,GSM5319618,GSM5319619,GSM5319620
ID,,,,,,,,,,,,,,,,,,,,,
cg00000029,0.702123,0.693371,0.651761,0.537299,0.697758,0.627569,0.626706,0.670315,0.668162,0.726896,...,0.709689,0.741576,0.656961,0.705426,0.642242,0.631936,0.717294,0.700089,0.663972,0.637874
cg00000108,0.940050,0.952013,0.933736,0.943061,0.944789,0.938310,0.952335,0.954303,0.946712,0.946946,...,0.954381,0.950791,0.941069,0.943805,0.936292,0.941607,0.924271,0.951583,0.949621,0.953637
cg00000109,0.870106,0.884304,0.852759,0.884553,0.886271,0.862609,0.898164,0.880870,0.902023,0.883341,...,0.817840,0.856216,0.895021,0.829832,0.890488,0.882086,0.836403,0.865641,0.851272,0.852899
cg00000165,0.214293,0.234931,0.234927,0.196809,0.210050,0.137444,0.208228,0.215354,0.190145,0.210846,...,0.191836,0.163157,0.203063,0.265601,0.208227,0.232426,0.211118,0.243064,0.251954,0.196533
cg00000236,0.830271,0.817191,0.819122,0.731896,0.827410,0.824601,0.796264,0.860995,0.841182,0.856055,...,0.847079,0.849355,0.712480,0.789161,0.822209,0.822677,0.821650,0.746469,0.790336,0.755776


In [15]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['cg00000029',
 'cg00000108',
 'cg00000109',
 'cg00000165',
 'cg00000236',
 'cg00000289',
 'cg00000292',
 'cg00000321',
 'cg00000363',
 'cg00000622',
 'cg00000658',
 'cg00000714',
 'cg00000721',
 'cg00000734',
 'cg00000769',
 'cg00000807',
 'cg00000884',
 'cg00000905',
 'cg00000924',
 'cg00000948']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [16]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['cg00000029', 'cg00000108', 'cg00000109', 'cg00000165', 'cg00000236', 'cg00000289', 'cg00000292', 'cg00000321', 'cg00000363', 'cg00000622', 'cg00000658', 'cg00000714', 'cg00000721', 'cg00000734', 'cg00000769', 'cg00000807', 'cg00000884', 'cg00000905', 'cg00000924', 'cg00000948']\n"

If not required, jump directly to the gene normalization step

In [17]:
requires_gene_mapping = True

In [18]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, 31717405.0, nan, nan, nan], 'AlleleB_ProbeSeq': [nan, 'ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAACCCCG', nan, nan, nan], 'Infinium_Design_Type': ['II', 'I', 'II', 'II', 'II'], 'Next_Base': [nan, 'A', nan, nan, nan], 'Color_Channel': [nan, 'Red', nan, nan, nan], 'Forward_Sequence': ['AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCTGCTCAGCTGGAAAC[CG]AATTTGAGATAAATTCAAGGGTCTATGTGGACAAGACTGCTAGTGTCTCTCTCTGGATTG', 'TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTG

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [19]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'Name': ['cg00035864', 'cg00050873', 'cg00061679', 'cg00063477', 'cg00121626'], 'AddressA_ID': ['31729416', '32735311', '28780415', '16712347', '19779393'], 'AlleleA_ProbeSeq': ['AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAAATTC', 'ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAACCCCA', 'AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTCCATC', 'TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTCCATC', 'AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAACTCTC'], 'AddressB_ID': [nan, 31717405.0, nan, nan, nan], 'AlleleB_P

In [20]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'UCSC_RefGene_Name'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [21]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

17 input query terms found dup hits:	[('ABCA11P', 2), ('ABCA17P', 2), ('ABCC13', 2), ('ABCC6P1', 2), ('ABCC6P2', 3), ('ADAM21P1', 2), ('A
72 input query terms found no hit:	['A2BP1', 'A2LD1', 'AACSL', 'AARS', 'ABP1', 'ACCN1', 'ACCN2', 'ACCN3', 'ACCN4', 'ACCN5', 'ACN9', 'AC
7 input query terms found dup hits:	[('ATP6AP1L', 2), ('ATXN8OS', 2), ('BAGE2', 2), ('BIRC8', 2), ('BMS1P1', 2), ('BMS1P4', 2), ('BRD7P3
365 input query terms found no hit:	['ARNTL2', 'ARPM1', 'ARPP-21', 'ARSE', 'ASAM', 'ASAP1IT1', 'ASFMR1', 'ASNA1', 'ATHL1', 'ATP5A1', 'AT
2 input query terms found dup hits:	[('C2orf83', 2), ('C3P1', 2)]
642 input query terms found no hit:	['C17orf56', 'C17orf57', 'C17orf59', 'C17orf60', 'C17orf61', 'C17orf62', 'C17orf63', 'C17orf64', 'C1
7 input query terms found dup hits:	[('CATSPER2P1', 2), ('CCDC144NL', 2), ('CCT6P1', 2), ('CDR1', 2), ('CECR7', 2), ('CLCA3P', 2), ('CLE
101 input query terms found no hit:	['CBARA1', 'CBWD1', 'CBWD2', 'CBWD3', 'CBWD5', 'CBWD6', 'CCBL1', 'CCBL2', 'C

In [22]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [23]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 32 samples.


In [24]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Intellectual-Disability', the least common label is '1.0' with 15 occurrences. This represents 46.88% of the dataset.
The distribution of the feature 'Intellectual-Disability' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.5
  50% (Median): 31.5
  75%: 40.25
Min: 6.0
Max: 56.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 12 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

A new JSON file was created at: ../../output1/Yuxuan/Intellectual-Disability/cohort_info.json


In [26]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [44]:
from utils import *

In [45]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE174555'

In [46]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE174555,True,True,False,True,True,32,


In [47]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [48]:
merged_data.head()

,Intellectual-Disability,Age,Gender,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0.0,37.0,1.0,0.590116,0.850843,0.799175,0.861421,0.412644,0.800985,0.833241,...,0.271460,0.101780,0.341449,0.471532,0.334077,0.557670,0.538319,0.351191,0.500098,0.198641
1,0.0,38.0,1.0,0.564490,0.878111,0.788008,0.860366,0.422047,0.794331,0.838132,...,0.269775,0.096716,0.303264,0.474052,0.345844,0.547580,0.520090,0.362980,0.499147,0.194963
2,0.0,38.0,0.0,0.587041,0.853657,0.795699,0.851574,0.418329,0.793030,0.825547,...,0.275054,0.110712,0.443794,0.541082,0.307189,0.511284,0.549419,0.322107,0.502943,0.194256
3,1.0,25.0,0.0,0.636154,0.852854,0.797266,0.843243,0.405989,0.759727,0.836518,...,0.265513,0.098102,0.455131,0.556389,0.330251,0.532812,0.559010,0.339559,0.494434,0.189841
4,0.0,30.0,1.0,0.574658,0.828747,0.782130,0.851762,0.425464,0.765987,0.817494,...,0.284233,0.106924,0.304203,0.453697,0.335481,0.519739,0.557386,0.342218,0.503108,0.199056


In [49]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [50]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [51]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [52]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [53]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 16.67% ± 14.91%


In [54]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 96.67% ± 6.67%


In [55]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [56]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0464
Found 27 genes with non-zero coefficients associated with the trait 'Intellectual-Disability' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
   ZNF141     9.949472              9.949472
   AMDHD1    -6.650276              6.650276
   ENDOD1    -6.558184              6.558184
KRTAP21-3     6.164865              6.164865
  FAM135A    -5.701479              5.701479
   STARD9    -5.205430              5.205430
    ZNF25    -4.786179              4.786179
    SMC1A    -4.750756              4.750756
    FARS2    -4.744686              4.744686
     USP1     4.735842              4.735842
    NLRP9     3.619005              3.619005
   CYP3A4     3.373809              3.373809
    AMACR    -2.977552              2.977552
    PWRN1    -2.913494              2.913494
     ALLC     2.860756              2.86075

/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [57]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0464
Found 74 significant genes associated with the trait 'Intellectual-Disability' conditional on the factor 'Age', with corrected p-value < 0.05:
  Variable  Coefficient      p_value  corrected_p_value
    RNF112    -0.361060 3.785914e-07           0.002083
     BCL9L     0.377690 5.369121e-07           0.002083
       NTS     0.409934 2.143989e-07           0.002083
    LRRC24     0.327457 5.836726e-07           0.002083
     CPT1B     0.326475 4.737830e-07           0.002083
    MAPK11     0.321815 1.295466e-06           0.003751
     CLIC3     0.362410 1.471685e-06           0.003751
CHKB-CPT1B     0.308444 1.911635e-06           0.004263
    RHBDL1     0.328942 3.153455e-06           0.005115
       AVP     0.317942 2.903821e-06           0.005115
      HHIP     0.332540 2.884898e-06           0.005115
     PRRT3     0.323225 3.986262e-06           0.005471
       GP6    -0.309213 3.925014e-06           

/home/yxcheng/AI4Science_gold_standard/stage1/Jianrong/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
